In [3]:
import dao.MongoTagUser as mtu
import panda.TagUser as tupd

mongotu = mtu.TagofSongUserRecom(user_id="1a78b764b67911ed95dd00155dadb10b")
# list = mongotu.makeRecomUsersSet()
# print(list)

In [4]:
doc = mongotu.rdb.findDocument(
            collection_name="like", query=mongotu.query, projection=mongotu.projections)
# 储存请求用户的名字
mongotu.user_name = doc[0]['name']
# 随机获取一定数量的用户来计算tag 相似度
# 统一获取其他用户的信息
querys = [{'$sample': {"size": 50-1}}]
project = {'$project': mongotu.a_projections}
querys.append(project)
docs = mongotu.rdb.aggregateDocument(
     collection_name="like", querys=querys)
print(doc)

[{'id': '1a78b764b67911ed95dd00155dadb10b', 'name': 'cLCH', 'tags': ['兴奋', '乡村', '金属', '经典', '民谣', '日语', '治愈', '爵士', '舞曲', '后摇', '游戏', '说唱', '小语种', '古典', '钢琴', 'Bossa Nova', '影视原声', '放松', '拉丁', '世界音乐', '蓝调', '夜晚', '伤感', '流行', '学习', '清新', '华语', '驾车', '90后', '浪漫', '酒吧', '另类/独立', '英伦', '孤独', '下午茶', '电子', '轻音乐', '器乐', '摇滚', '民族', 'R&B/Soul', '粤语', '安静', '运动', '旅行', '雷鬼', '感动', '怀旧', '快乐', '朋克', '韩语', '吉他', '欧美', 'ACG', '古风', 'New Age']}]


In [5]:
# 判断 docs中是否存在 doc 并合并
docs = mongotu.isDocinDocs(doc=doc, docs=docs)
# print(docs)     

In [6]:
# 查找数据songs 便于 统计tags 出现的次数
# 提取docs 中用户的id，之后用
list_name = mongotu.getItemFromDocs(docs=docs, key='name')
list_id = mongotu.getItemFromDocs(docs=docs, key='id')
# 通过这个方法获取songs，来计算tag出现次数
docs_song = mongotu.getSongsforTagfromMongo(list_id)
# count_tag 统计
dict_tags = mongotu.makeMatricTagCountData(docs=docs_song)
# print(dict_tags)
        

In [7]:
# 根据以上docs 来生成 xy轴，矩阵
# 内容
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongotu.makeXYMatric(docs=docs)
# print(data)
        

In [8]:
# 生成pandas 表格
tudf = tupd.TUPandas(datas=list(
map(list, data.toarray())), items=mongotu.y, users=mongotu.x)



In [9]:
tudf.df 

,cLCH,eVbz,G,8t9T0zOQpl,2YjBEkHgK,D0nHKfvH,I3F0KrG3a,EQAFph8,m3NOW7Pp,HCV4F,...,dj775w,MGrVl,i7qZ,w,9D0ZA,wxVuBFlbYW,520a2,2ILOvc,b,KvC
日语,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
英伦,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
古典,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
后摇,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
影视原声,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
下午茶,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
摇滚,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
游戏,1,1,1,0,1,0,0,0,1,1,...,1,1,1,1,1,0,1,1,1,1
ACG,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [10]:
# 开始将tudf 中的数据根据上面的dict_tags中的数据进行替换
tudf = mongotu.changeDataCountTagTUDF(list_name, dict_tags, tudf)
   

In [11]:
tudf.df     

,cLCH,eVbz,G,8t9T0zOQpl,2YjBEkHgK,D0nHKfvH,I3F0KrG3a,EQAFph8,m3NOW7Pp,HCV4F,...,dj775w,MGrVl,i7qZ,w,9D0ZA,wxVuBFlbYW,520a2,2ILOvc,b,KvC
日语,66,18,55,6,49,10,35,46,62,65,...,34,89,36,50,63,5,13,41,41,43
英伦,34,17,29,4,22,8,16,21,24,24,...,16,40,24,22,22,6,5,20,20,23
古典,37,15,27,6,21,9,20,27,39,41,...,27,51,21,30,43,8,13,34,21,32
后摇,19,12,21,3,19,4,12,15,38,33,...,13,28,28,11,32,7,11,22,16,15
影视原声,66,26,58,7,44,10,21,44,72,65,...,27,72,44,42,71,7,12,41,45,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
下午茶,2,1,2,1,3,0,1,2,5,2,...,2,2,1,6,7,1,1,3,2,3
摇滚,104,52,84,13,66,24,48,77,114,118,...,51,109,72,90,100,13,26,75,58,72
游戏,4,2,4,0,4,0,0,0,2,4,...,4,8,4,1,1,0,2,3,5,4
ACG,85,26,70,8,51,17,42,61,92,93,...,47,101,58,53,76,6,18,56,55,62


In [12]:
# 推荐过程
similar_user = tudf.makeSimilarityWithPearson()

In [13]:
similar_user

,cLCH,eVbz,G,8t9T0zOQpl,2YjBEkHgK,D0nHKfvH,I3F0KrG3a,EQAFph8,m3NOW7Pp,HCV4F,...,dj775w,MGrVl,i7qZ,w,9D0ZA,wxVuBFlbYW,520a2,2ILOvc,b,KvC
cLCH,1.00,0.97,0.99,0.94,0.98,0.96,0.97,0.98,0.98,0.98,...,0.98,0.98,0.98,0.97,0.98,0.91,0.94,0.98,0.98,0.99
eVbz,0.97,1.00,0.95,0.92,0.95,0.94,0.94,0.96,0.96,0.96,...,0.95,0.95,0.95,0.95,0.96,0.94,0.94,0.96,0.96,0.96
G,0.99,0.95,1.00,0.91,0.98,0.96,0.96,0.97,0.97,0.98,...,0.97,0.98,0.97,0.97,0.98,0.89,0.93,0.97,0.98,0.98
8t9T0zOQpl,0.94,0.92,0.91,1.00,0.92,0.94,0.94,0.95,0.93,0.93,...,0.95,0.92,0.94,0.93,0.93,0.92,0.92,0.93,0.91,0.93
2YjBEkHgK,0.98,0.95,0.98,0.92,1.00,0.95,0.97,0.97,0.98,0.97,...,0.97,0.98,0.98,0.97,0.97,0.90,0.96,0.96,0.98,0.97
D0nHKfvH,0.96,0.94,0.96,0.94,0.95,1.00,0.96,0.96,0.95,0.95,...,0.97,0.95,0.97,0.96,0.96,0.93,0.95,0.95,0.95,0.97
I3F0KrG3a,0.97,0.94,0.96,0.94,0.97,0.96,1.00,0.98,0.97,0.97,...,0.98,0.97,0.96,0.97,0.96,0.92,0.95,0.96,0.95,0.97
EQAFph8,0.98,0.96,0.97,0.95,0.97,0.96,0.98,1.00,0.98,0.98,...,0.98,0.98,0.97,0.98,0.98,0.94,0.95,0.97,0.97,0.98
m3NOW7Pp,0.98,0.96,0.97,0.93,0.98,0.95,0.97,0.98,1.00,0.99,...,0.98,0.98,0.98,0.97,0.98,0.93,0.96,0.97,0.97,0.98
HCV4F,0.98,0.96,0.98,0.93,0.97,0.95,0.97,0.98,0.99,1.00,...,0.98,0.98,0.97,0.98,0.99,0.92,0.95,0.98,0.97,0.98


In [14]:
# 获取相似度最高几位用户sign 设置为 1 表示 使用Pearson相关系数获取相关性最高的几个用户
diction = tudf.makeTopNUsers(similar=similar_user,sign=1)
print(diction)
print(len(diction['RuRuUIH']))

{'cLCH': ['KvC', 'p', 'G', 'GOUkTpak', 'zAaptCORtu', 'S0JlP3z', 'b', '2ILOvc', '9D0ZA', 'i7qZ', 'MGrVl', 'dj775w', 'BrQ', 'fRcKBK16j', 'mwGsn4', 'FnJi', 'N2LBNEz', 'ReOewQ', '2YjBEkHgK', 'EQAFph8', 'm3NOW7Pp', 'HCV4F', 'dyrvSYZxx', '6p11UivDhF', 'X2Bwh38', 'RuRuUIH', 'w', 'I3F0KrG3a', 'eVbz', 'HrlIEqGS', 'Qu0', 'J94COPc', 'WEZ3ns0jN', '7ljkMbQeu', 'afqmIx79w', 'BK1076n', 'D0nHKfvH']}


KeyError: 'RuRuUIH'

In [15]:
# 利用当前用户的信息确定他们的id值形成对应的list_id
diction[mongotu.user_name] = mongotu.makeListIdWithUserName(diction[mongotu.user_name])
print(diction)
print(len(diction['RuRuUIH']))

{'cLCH': ['1a7904e4b67911ed95dd00155dadb10b', '1a78b656b67911ed95dd00155dadb10b', '1a78b0c0b67911ed95dd00155dadb10b', '1a78ee28b67911ed95dd00155dadb10b', '1a7922eeb67911ed95dd00155dadb10b', '1a790340b67911ed95dd00155dadb10b', '1a7900f2b67911ed95dd00155dadb10b', '1a78bbceb67911ed95dd00155dadb10b', '1a78b3d6b67911ed95dd00155dadb10b', '1a78cd12b67911ed95dd00155dadb10b', '1a791894b67911ed95dd00155dadb10b', '1a792000b67911ed95dd00155dadb10b', '1a78f3f0b67911ed95dd00155dadb10b', '1a78c22cb67911ed95dd00155dadb10b', '1a78d578b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a78e95ab67911ed95dd00155dadb10b', '1a78f15cb67911ed95dd00155dadb10b', '1a79179ab67911ed95dd00155dadb10b', '1a78bfcab67911ed95dd00155dadb10b', '1a78ba0cb67911ed95dd00155dadb10b', '1a78c74ab67911ed95dd00155dadb10b', '1a79064cb67911ed95dd00155dadb10b', '1a78c6a0b67911ed95dd00155dadb10b', '1a78bb2eb67911ed95dd00155dadb10b', '1a78ac10b67911ed95dd00155dadb10b', '1a78f332b67911ed95dd00155dadb10b', '1a78c3d0b67911ed9

KeyError: 'RuRuUIH'

In [ ]:
# 保存过程
# mongotu.saveDataSetRecomAnswer(diction)
# 改变形式，方便存储
docs = mongotu.changeDataFormat(diction=diction, list_keys=list(diction))
print(docs)
# 保存
mongotu.wdb.writeDocument(docs=docs, collection_name="recom")

[{'id': '1a78ac10b67911ed95dd00155dadb10b', 'name': 'RuRuUIH', 'RecomUsers': ['1a78b656b67911ed95dd00155dadb10b', '1a78e1eeb67911ed95dd00155dadb10b', '1a7904e4b67911ed95dd00155dadb10b', '1a78ba8eb67911ed95dd00155dadb10b', '1a78c6a0b67911ed95dd00155dadb10b', '1a78f15cb67911ed95dd00155dadb10b', '1a790b06b67911ed95dd00155dadb10b', '1a792000b67911ed95dd00155dadb10b', '1a78bbceb67911ed95dd00155dadb10b', '1a7909a8b67911ed95dd00155dadb10b', '1a78ae7cb67911ed95dd00155dadb10b', '1a791bf0b67911ed95dd00155dadb10b', '1a78b80eb67911ed95dd00155dadb10b', '1a790340b67911ed95dd00155dadb10b', '1a78de4cb67911ed95dd00155dadb10b', '1a7900f2b67911ed95dd00155dadb10b', '1a78b962b67911ed95dd00155dadb10b', '1a790d2cb67911ed95dd00155dadb10b', '1a78f4a4b67911ed95dd00155dadb10b', '1a78d3e8b67911ed95dd00155dadb10b', '1a791aceb67911ed95dd00155dadb10b', '1a78d28ab67911ed95dd00155dadb10b', '1a78d046b67911ed95dd00155dadb10b', '1a78e4b4b67911ed95dd00155dadb10b', '1a78dd66b67911ed95dd00155dadb10b', '1a79134eb67911ed95dd0